In [4]:
repo_url = 'https://github.com/ZZIDZZ/tensorflow-object-detection.git'
# Number of training steps - 1000 will train very quickly, but more steps will increase accuracy.
num_steps = 10000  # 200000 to improve

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    }
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'ssd_mobilenet_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'D:\\Projects\\pyproj\\models\\research\\pretrained_model'

In [13]:
!dir

KeyError: 'PYTHONPATH'

In [12]:
!python generate_tfrecord.py --csv_input data\annotations --img_path data\train

Traceback (most recent call last):
  File "d:\Projects\pyproj\tensorflow-object-detection\generate_tfrecord.py", line 134, in <module>
    tf.app.run()
  File "D:\Projects\pyproj\project_env\lib\site-packages\tensorflow\python\platform\app.py", line 36, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "D:\Projects\pyproj\project_env\lib\site-packages\absl\app.py", line 312, in run
    _run_main(main, args)
  File "D:\Projects\pyproj\project_env\lib\site-packages\absl\app.py", line 258, in _run_main
    sys.exit(main(argv))
  File "d:\Projects\pyproj\tensorflow-object-detection\generate_tfrecord.py", line 107, in main
    writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
  File "D:\Projects\pyproj\project_env\lib\site-packages\tensorflow\python\lib\io\tf_record.py", line 294, in __init__
    super(TFRecordWriter, self).__init__(
tensorflow.python.framework.errors_impl.NotFoundError: Failed to create a NewWriteableFile:  : The system cannot fin

In [14]:
!where python

D:\Projects\pyproj\project_env\Scripts\python.exe
C:\Users\gamer\AppData\Local\Programs\Python\Python39\python.exe


In [20]:
%D:\Projects\pyproj\project_env\Scripts\activate

UsageError: Line magic function `%D:\Projects\pyproj\project_env\Scripts\activate` not found.


In [21]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile


if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

[WinError 3] The system cannot find the path specified: '/content/models/research'
d:\Projects\pyproj\tensorflow-object-detection


In [5]:
%cd D:\\Projects\\pyproj\\models\\research
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

D:\Projects\pyproj\models\research


'D:\\Projects\\pyproj\\models\\research\\pretrained_model\\model.ckpt'

In [9]:
import os
pipeline_fname = os.path.join('D:\\Projects\\pyproj\\models\\research\\object_detection\\samples\\configs\\', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [6]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [14]:
import re
train_record_fname = 'D:\\Projects\\pyproj\\tensorflow-object-detection\\train.record'
test_record_fname = 'D:\\Projects\\pyproj\\tensorflow-object-detection\\test.record'
label_map_pbtxt_fname = 'D:\\Projects\\pyproj\\tensorflow-object-detection\\ppe\\label_map.pbtxt'
num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

error: bad escape \P at position 25